In [4]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 30.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.11.1
    Uninstalling huggingface-hub-0.11.1:
      Successfully uninstalled huggingface-hub-0.11.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.25.1
    Uninstalling transformers-4.25.1:
      Successfully uninstalled transformers-4.25.1

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 6.3 MB/s eta 0:00:00m eta 0:00:010:00:01m

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [46]:
from transformers import AutoImageProcessor, TimesformerForVideoClassification, TimesformerModel, AutoModel, AutoProcessor
import numpy as np
import torch
import av
import torch
import numpy as np
import requests

from huggingface_hub import hf_hub_download

# Classification

In [7]:
np.random.seed(0)


def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


In [22]:
def predict(file_path, num_frames=8):

    container = av.open(file_path)

    # sample 8 frames
    indices = sample_frame_indices(clip_len=num_frames, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
    video = read_video_pyav(container, indices)

    image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
    model = TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400")

    inputs = image_processor(list(video), return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # model predicts one of the 400 Kinetics-400 classes
    predicted_label = logits.argmax(-1).item()
    print(model.config.id2label[predicted_label])

In [44]:
# video clip consists of 300 frames (10 seconds at 30 FPS)
demo_video = hf_hub_download(
    repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"
)

In [45]:
%%time
predict(demo_video)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


eating spaghetti
CPU times: user 7.67 s, sys: 1.19 s, total: 8.86 s
Wall time: 7.48 s


In [24]:
pigeon_url = 'https://cdn.fosstodon.org/cache/media_attachments/files/110/612/356/850/889/602/original/c897f30545d84ff8.mp4'

In [25]:
football_url = 'https://cdn.fosstodon.org/cache/media_attachments/files/110/612/427/994/356/928/original/ff329baeabe24f2c.mp4'

In [34]:
# music video with announcement of a concert
music_url = 'https://cdn.fosstodon.org/cache/media_attachments/files/110/612/945/969/492/367/original/a76449edc5b7a016.mp4'

In [29]:
predict(pigeon_url, num_frames=16)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


feeding birds


In [32]:
predict(football_url)


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


passing American football (not in game)


In [35]:
predict(music_url)


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


answering questions


# Extracting features

In [39]:
def extract(url):
    container = av.open(file_path)

    # sample 8 frames

    indices = sample_frame_indices(clip_len=8, frame_sample_rate=4, seg_len=container.streams.video[0].frames)
    video = read_video_pyav(container, indices)

    image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
    model = TimesformerModel.from_pretrained("facebook/timesformer-base-finetuned-k400")

    # prepare video for the model
    inputs = image_processor(list(video), return_tensors="pt")

    # forward pass
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    print(list(last_hidden_states.shape))

In [40]:
extract(football_url)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of the model checkpoint at facebook/timesformer-base-finetuned-k400 were not used when initializing TimesformerModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TimesformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TimesformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[1, 1569, 768]


In [41]:
extract(music_url)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of the model checkpoint at facebook/timesformer-base-finetuned-k400 were not used when initializing TimesformerModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TimesformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TimesformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[1, 1569, 768]


# Zero shot classification

In [47]:
def predict_zeroshot(file_path, texts):
    container = av.open(file_path)

    # sample 8 frames
    indices = sample_frame_indices(clip_len=8, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
    video = read_video_pyav(container, indices)

    processor = AutoProcessor.from_pretrained("microsoft/xclip-base-patch32")
    model = AutoModel.from_pretrained("microsoft/xclip-base-patch32")

    inputs = processor(
        text=texts,
        videos=list(video),
        return_tensors="pt",
        padding=True,
    )

    # forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    logits_per_video = outputs.logits_per_video  # this is the video-text similarity score
    probs = logits_per_video.softmax(dim=1)  # we can take the softmax to get the label probabilities

    print(f'Logits: {logits_per_video}')
    print(f'Probs: {[round(prob, 4) for prob in probs.detach().numpy().tolist()[0]]}')

In [48]:
predict_zeroshot(demo_video, ["playing sports", "eating spaghetti", "go shopping"])

Logits: tensor([[10.8215, 19.3644, 11.6707]])
Probs: [0.0002, 0.9993, 0.0005]


In [49]:
predict_zeroshot(football_url, ["sports", "music", "animals", "harmful", "concert", "announcement"])

Logits: tensor([[16.4240, 14.9282, 15.0922, 14.9156, 13.9682, 14.3310]])
Probs: [0.5212, 0.1168, 0.1376, 0.1153, 0.0447, 0.0643]


In [50]:
predict_zeroshot(music_url, ["sports", "music", "animals", "harmful", "concert", "announcement"])

Logits: tensor([[ 9.8448, 12.4687, 11.5755, 10.5263, 12.1523, 10.7949]])
Probs: [0.0285, 0.3935, 0.1611, 0.0564, 0.2867, 0.0738]


In [51]:
predict_zeroshot(pigeon_url, ["sports", "music", "animals", "harmful", "concert", "announcement"])

Logits: tensor([[20.4419, 20.2537, 22.4075, 19.5911, 19.4784, 20.6230]])
Probs: [0.0911, 0.0755, 0.6505, 0.0389, 0.0348, 0.1092]
